# 2クラスのマルチクラス分類

In [0]:
from keras.datasets import cifar10

import numpy as np
import keras
import tensorflow as tf
import copy
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers import Input
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

random_state = 42

In [0]:
# 保存ディレクトリを作成
saveDir = "./multi2cifar10/"
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)

In [0]:
batch_size = 50
num_classes1 = 10
num_classes2 = 2
epochs = 10

In [0]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

In [0]:
#画像を0-1の範囲で正規化
x_train=x_train.astype('float32')/255.0
x_test=x_test.astype('float32')/255.0


In [0]:
# 各ラベルに名前を設定
label1 = {0:"airplane",
          1:"automobile",
          2:"bird",
          3:"cat",
          4:"deer",
          5:"dog",
          6:"frog",
          7:"horse",
          8:"ship",
          9:"truck"}
label2 = {0:"artifact", #0,1,8,9
          1:"animal"}   #2,3,4,5,6,7

In [0]:
# 人工物を0、動物を1として、y_train2とy_test2を定義
def modLabel2(y):
    y2 = copy.deepcopy(y)
    for i in range(len(y)):
        if y2[i] in [0,1,8,9]:
            y2[i] = 0
        else:
            y2[i] = 1
    return y2

In [0]:
y_train2 = modLabel2(y_train)
y_test2 = modLabel2(y_test)

In [0]:
# クラスベクトルをバイナリクラス行列に変換
y_train1 = keras.utils.to_categorical(y_train, num_classes1)
y_test1 = keras.utils.to_categorical(y_test, num_classes1)

y_train2 = keras.utils.to_categorical(y_train2, num_classes2)
y_test2 = keras.utils.to_categorical(y_test2, num_classes2)
    
    
print("y train1:{0}\n y test1:{1}\n y train2:{2}\n y test2:{3}\n".format(
    y_train1.shape, y_test1.shape, y_train2.shape, y_test2.shape))

for i in range(10):
    plt.figure(figsize=(2, 2))
    print("data{0} is {1} and {2} ".format(
        i,label1[np.argmax(y_train1[i])], label2[np.argmax(y_train2[i])]))
    plt.imshow(x_train[i].reshape(32, 32, 3))
    plt.show()

In [0]:
from keras import layers
from keras import models
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten

# 入力層
model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'
          ,input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model_output = model.output

output1 = Dense(num_classes1, activation='softmax', name='output1')(model_output)
output2 = Dense(num_classes2, activation='softmax', name='output2')(model_output)

multiModel = Model(inputs=model.input, outputs=[output1, output2])

In [0]:
# Adamオプティマイザを設定
opt = keras.optimizers.adam(lr=0.0001, decay=1e-6)

# Adamを使ってモデルを訓練
multiModel.compile(loss={'output1': 'categorical_crossentropy',
                         'output2': 'categorical_crossentropy'},
                   optimizer=opt,
                   metrics=['accuracy'])

In [0]:
# early stopping と modelのcheckpoint
es_cb = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
chkpt = os.path.join(saveDir, 'MultiCifar10_.{epoch:02d}-{val_loss:.2f}- \
                     {val_output1_loss:.2f}-{val_output2_loss:.2f}.hdf5')
cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', 
                        verbose=1, save_best_only=True, mode='auto')

history = multiModel.fit(x_train,
                         {'output1': y_train1,
                          'output2': y_train2},
                         batch_size=batch_size,
                         epochs=epochs,
                         validation_data=(x_test,
                                          {'output1': y_test1,
                                           'output2': y_test2}),
                         callbacks=[es_cb, cp_cb])

30550/50000 [=================>............] - ETA: 7s - loss: 1.7695 - output1_loss: 1.5081 - output2_loss: 0.2614 - output1_acc: 0.4464 - output2_acc: 0.8955

In [0]:
scores = multiModel.evaluate(x_test,
                            {'output1': y_test1,
                             'output2': y_test2}, batch_size=batch_size)

In [0]:
print("total loss:{0}".format(scores[0]))
print("label1 loss:{0}\taccuracy:{1}:".format(scores[1],scores[3]))
print("label2 loss:{0}\taccuracy:{1}:".format(scores[2],scores[4]))

# 3クラスのマルチクラス分類

In [0]:
from keras.datasets import cifar10

import numpy as np
import keras
import tensorflow as tf
import copy
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers import Input
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

random_state = 42

In [0]:
# 保存ディレクトリを作成
saveDir = "./multi3cifar10/"
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)